In [ ]:
!pip -q uninstall -y pillow Pillow || true
!pip -q install --upgrade --force-reinstall "pillow<12.0"
!pip -q install --upgrade diffusers transformers accelerate safetensors huggingface_hub opencv-python

import os, math, random
import torch
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFilter
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionInpaintPipeline,
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def to_grid(images, cols=2, bg=255):
    if isinstance(images, Image.Image):
        images = [images]
    w, h = images[0].size
    rows = math.ceil(len(images) / cols)
    grid = Image.new("RGB", (cols*w, rows*h), (bg, bg, bg))
    for i, im in enumerate(images):
        grid.paste(im, ((i % cols)*w, (i // cols)*h))
    return grid

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32
print("device:", device, "| dtype:", dtype)

In [ ]:
seed_everything(7)
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=dtype,
    safety_checker=None,
).to(device)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

if device == "cuda":
    pipe.enable_attention_slicing()
    pipe.enable_vae_slicing()

prompt = "a cinematic photo of a futuristic street market at dusk, ultra-detailed, 35mm, volumetric lighting"
negative_prompt = "blurry, low quality, deformed, watermark, text"

img_text = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=25,
    guidance_scale=6.5,
    width=768,
    height=512,
).images[0]

In [ ]:
LCM_LORA = "latent-consistency/lcm-lora-sdv1-5"
pipe.load_lora_weights(LCM_LORA)

try:
    pipe.fuse_lora()
    lora_fused = True
except Exception as e:
    lora_fused = False
    print("LoRA fuse skipped:", e)

fast_prompt = "a clean product photo of a minimal smartwatch on a reflective surface, studio lighting"
fast_images = []
for steps in [4, 6, 8]:
    fast_images.append(
        pipe(
            prompt=fast_prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps,
            guidance_scale=1.5,
            width=768,
            height=512,
        ).images[0]
    )

grid_fast = to_grid(fast_images, cols=3)
print("LoRA fused:", lora_fused)

W, H = 768, 512
layout = Image.new("RGB", (W, H), "white")
draw = ImageDraw.Draw(layout)
draw.rectangle([40, 80, 340, 460], outline="black", width=6)
draw.ellipse([430, 110, 720, 400], outline="black", width=6)
draw.line([0, 420, W, 420], fill="black", width=5)

edges = cv2.Canny(np.array(layout), 80, 160)
edges = np.stack([edges]*3, axis=-1)
canny_image = Image.fromarray(edges)

CONTROLNET = "lllyasviel/sd-controlnet-canny"
controlnet = ControlNetModel.from_pretrained(
    CONTROLNET,
    torch_dtype=dtype,
).to(device)

cn_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL,
    controlnet=controlnet,
    torch_dtype=dtype,
    safety_checker=None,
).to(device)

cn_pipe.scheduler = UniPCMultistepScheduler.from_config(cn_pipe.scheduler.config)

if device == "cuda":
    cn_pipe.enable_attention_slicing()
    cn_pipe.enable_vae_slicing()

cn_prompt = "a modern cafe interior, architectural render, soft daylight, high detail"
img_controlnet = cn_pipe(
    prompt=cn_prompt,
    negative_prompt=negative_prompt,
    image=canny_image,
    num_inference_steps=25,
    guidance_scale=6.5,
    controlnet_conditioning_scale=1.0,
).images[0]

In [2]:
mask = Image.new("L", img_controlnet.size, 0)
mask_draw = ImageDraw.Draw(mask)
mask_draw.rectangle([60, 90, 320, 170], fill=255)
mask = mask.filter(ImageFilter.GaussianBlur(2))

inpaint_pipe = StableDiffusionInpaintPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=dtype,
    safety_checker=None,
).to(device)

inpaint_pipe.scheduler = UniPCMultistepScheduler.from_config(inpaint_pipe.scheduler.config)

if device == "cuda":
    inpaint_pipe.enable_attention_slicing()
    inpaint_pipe.enable_vae_slicing()

inpaint_prompt = "a glowing neon sign that says 'CAFÉ', cyberpunk style, realistic lighting"

img_inpaint = inpaint_pipe(
    prompt=inpaint_prompt,
    negative_prompt=negative_prompt,
    image=img_controlnet,
    mask_image=mask,
    num_inference_steps=30,
    guidance_scale=7.0,
).images[0]

os.makedirs("outputs", exist_ok=True)
img_text.save("outputs/text2img.png")
grid_fast.save("outputs/lora_fast_grid.png")
layout.save("outputs/layout.png")
canny_image.save("outputs/canny.png")
img_controlnet.save("outputs/controlnet.png")
mask.save("outputs/mask.png")
img_inpaint.save("outputs/inpaint.png")

print("Saved outputs:", sorted(os.listdir("outputs")))
print("Done.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 112.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


device: cpu | dtype: torch.float32


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_inpaint.StableDiffusionInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or aud

  0%|          | 0/30 [00:00<?, ?it/s]

Saved outputs: ['canny.png', 'controlnet.png', 'inpaint.png', 'layout.png', 'lora_fast_grid.png', 'mask.png', 'text2img.png']
Done.
